<a href="https://colab.research.google.com/github/dbasu03/copilot-gen-poc1/blob/main/Untitled15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import files
import pandas as pd




In [20]:
df = pd.read_csv('/content/Workflowinfo.csv' , encoding='ISO-8859-1')

In [21]:
df_filtered = df[df['WorkflowName'].str[0].str.isupper()]

df_filtered.to_csv('FilteredWorkflowinfo.csv', index=False)



In [38]:
files.download('FilteredWorkflowinfo.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
filtered_workflow_df = pd.read_csv('FilteredWorkflowinfo.csv')

In [41]:
nodeinfo_df = pd.read_csv('nodeinfo.csv', delimiter=';')
nodeinfo_df = pd.read_csv('nodeinfo.csv', on_bad_lines='skip')


filtered_workflow_ids = filtered_workflow_df['WorkflowId'].tolist()

filtered_nodeinfo_df = nodeinfo_df[nodeinfo_df['WorkflowId'].isin(filtered_workflow_ids)]

filtered_nodeinfo_df.to_csv('FilteredNodeinfo.csv', index=False)

In [42]:
files.download('FilteredNodeinfo.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
linkinfo = pd.read_csv('linkinfo.csv')
nodeinfo = pd.read_csv('nodeinfo.csv',delimiter=';')
nodeinfo = pd.read_csv('nodeinfo.csv', on_bad_lines='skip')

merged_from = pd.merge(linkinfo, nodeinfo, left_on='FromNodeId', right_on='WorkflowNodeId', how='left')
merged_full = pd.merge(merged_from, nodeinfo, left_on='ToNodeId', right_on='WorkflowNodeId', how='left')

result = merged_full[['NodeText_x', 'NodeText_y']]
result.columns = ['FromNodeName', 'ToNodeName']

result.to_csv('output.csv', index=False)

In [47]:
import pandas as pd


linkinfo_df = pd.read_csv('linkinfo.csv')
nodeinfo_df = pd.read_csv('nodeinfo.csv')

from_node_merged = linkinfo_df.merge(nodeinfo_df, left_on='FromNodeId', right_on='WorkflowNodeId', how='left')
to_node_merged = from_node_merged.merge(nodeinfo_df, left_on='ToNodeId', right_on='WorkflowNodeId', how='left', suffixes=('_From', '_To'))


output_df = to_node_merged[['WorkflowId', 'NodeText_From', 'NodeText_To']]
output_df.rename(columns={'NodeText_From': 'FromNodeName', 'NodeText_To': 'ToNodeName'}, inplace=True)


output_df.to_csv('output.csv', index=False)


<ipython-input-47-51d4b5a1e818>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df.rename(columns={'NodeText_From': 'FromNodeName', 'NodeText_To': 'ToNodeName'}, inplace=True)


In [48]:
filtered_workflows = pd.read_csv('FilteredWorkflowinfo.csv')

filtered_workflow_ids = filtered_workflows['WorkflowId'].unique()

output_df = pd.read_csv('output.csv')

filtered_output_df = output_df[output_df['WorkflowId'].isin(filtered_workflow_ids)]

filtered_output_df.to_csv('FilteredLinkinfo.csv', index=False)

In [49]:
files.download('FilteredLinkinfo.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [50]:
filtered_workflow_info = pd.read_csv('FilteredWorkflowinfo.csv')
filtered_node_info = pd.read_csv('FilteredNodeinfo.csv')
filtered_link_info = pd.read_csv('FilteredLinkinfo.csv')

workflows = {}

workflow_ids = filtered_workflow_info['WorkflowId'].unique()

for workflow_id in workflow_ids:
    links = filtered_link_info[filtered_link_info['WorkflowId'] == workflow_id]

    node_dict = {}

    for _, row in links.iterrows():
        from_node = row['FromNodeName']
        to_node = row['ToNodeName']
        if from_node not in node_dict:
            node_dict[from_node] = []
        node_dict[from_node].append(to_node)

    workflow = ['Start']
    current_node = 'Start'
    visited_nodes = set()

    while current_node != 'End':
        visited_nodes.add(current_node)
        if current_node in node_dict:
            next_nodes = node_dict[current_node]
            next_node = None

            for node in next_nodes:
                if node not in visited_nodes:
                    next_node = node
                    break

            if next_node is None:
                print(f"Warning: Possible loop detected in workflow {workflow_id} at node {current_node}")
                break

            workflow.append(next_node)
            current_node = next_node
        else:
            print(f"Warning: No valid link found for node {current_node} in workflow {workflow_id}")
            break

    if workflow[-1] != 'End':
        print(f"Warning: 'End' node not found in workflow {workflow_id}. Generating workflow without 'End'.")

    workflows[workflow_id] = ' --> '.join(workflow)

workflow_df = pd.DataFrame(list(workflows.items()), columns=['WorkflowId', 'Workflow'])

workflow_df.to_csv('OrderedWorkflows.csv', index=False)


Streaming output truncated to the last 5000 lines.


In [33]:
files.download('OrderedWorkflows.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
file_path = '/content/OrderedWorkflows.csv'
df = pd.read_csv(file_path)

def generate_prompt(workflow):
    if "Shopify" in workflow and "ERP" in workflow:
        return "Generate a workflow for integrating Shopify with an ERP system for seamless order management."
    elif "Magento" in workflow and "Payment" in workflow:
        return "Create a workflow for managing payments in a Magento-based e-commerce store."
    elif "WooCommerce" in workflow and "Shipping" in workflow:
        return "Design a workflow for managing shipping in a WooCommerce store."
    else:
        return "Create a workflow for managing the business operations with the mentioned apps."

df['Prompt'] = df['Workflow'].apply(generate_prompt)

output_file_path = '/content/OrderedWorkflows_with_Prompt.csv'
df.to_csv(output_file_path, index=False)



In [54]:
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(df['Workflow'].tolist())


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [53]:
!pip install faiss-cpu
!pip install faiss-gpu


In [55]:
import faiss
import numpy as np

embeddings_np = np.array(embeddings)

dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)

index.add(embeddings_np)


In [56]:
faiss.write_index(index, 'faiss_index.index')

index = faiss.read_index('faiss_index.index')

In [57]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

df = pd.read_csv('OrderedWorkflows.csv')

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(df['Workflow'].tolist())

embeddings_np = np.array(embeddings)

dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)

index.add(embeddings_np)

faiss.write_index(index, 'faiss_index.index')

index = faiss.read_index('faiss_index.index')

user_input = input("Enter your text: ")

input_embedding = model.encode([user_input])[0]

k = 10
distances, indices = index.search(np.array([input_embedding]), k)

matching_workflows = df.iloc[indices[0]]['Workflow'].tolist()

print("Matching Workflows:")
for workflow in matching_workflows:
    print(workflow)


Enter your text: optimize sales process by integrating sap b1 and shopify ecommerce
Matching Workflows:
Start --> SAP B1 --> Mapper --> Shopify
Start --> SAP B1 --> Mapper --> Shopify
Start --> SAP B1 --> Mapper --> Shopify
Start --> SAP B1 --> Mapper --> Shopify
Start --> SAP B1 --> Mapper --> Shopify
Start --> SAP B1 --> Mapper --> Shopify
Start --> SAP B1 --> Mapper --> Shopify
Start --> SAP B1 --> Mapper --> Shopify
Start --> SAP B1 --> Mapper --> Shopify
Start --> SAP B1 --> Mapper --> Shopify


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

df = pd.read_csv('OrderedWorkflows.csv')

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(df['Workflow'].tolist())

embeddings_np = np.array(embeddings)

dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)

index.add(embeddings_np)

faiss.write_index(index, 'faiss_index.index')

index = faiss.read_index('faiss_index.index')

user_input = input("Enter your text: ")

input_embedding = model.encode([user_input])[0]


k = 5
distances, indices = index.search(np.array([input_embedding]), k)

matching_index = indices[0][0]

matching_workflow = df.iloc[matching_index]['Workflow']

print(matching_workflow)


In [28]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('OrderedWorkflows.csv')

# Define the workflow you're searching for
target_workflow = "Start --> SAP B1 --> Mapper --> Shopify"

# Find the row where the 'Workflow' column matches the target_workflow
matching_row = df[df['Workflow'] == target_workflow]

# Check if a match was found
if not matching_row.empty:
    # Extract the WorkflowId
    matching_workflow_id = matching_row['WorkflowId'].values[0]
    print(f"Matching WorkflowId: {matching_workflow_id}")
else:
    print("No matching workflow found.")


Matching WorkflowId: 098F596D-7052-41D4-A5E5-0E3E738C132D
